In [ ]:
import torch

A = torch.arange(10)
M = A % 2 == 0
B = torch.full_like(A, 10)[M]

# Can I do this out of place?
A[M] = B
A
# is there a function like C = foo(A, M, B)
# note that A and B don't have the same shape, so I can't use torch.where


def combine_mask(A, M1, M2):
    C = torch.full_like(A, False)
    C[M] = True
    C[M][~M2] = False
    return C

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchlensmaker as tlm

surface = tlm.surfaces.Sphere(diameter=15, r=tlm.parameter(18))
lens = tlm.BiLens(surface, material="SF10-nd", outer_thickness=1.5)

optics = nn.Sequential(
    tlm.Offset(
        tlm.Rotate(
            tlm.RaySource(),
            [-55, 0]),
        y=17.9),
    tlm.Gap(10),
    lens,
    tlm.Gap(30),
    tlm.FocalPoint(),
)

lens.optics[2].critical_angle = "drop"
# clamp works, drop doesn't -> need to handle drop in refractive surface - filter dropped rays

tlm.show(optics, dim=2, sampling={"base": 10})

In [ ]:
### BUG V1
# happens no matter when the critical_angle argument is

import torch
import torch.nn as nn
import torch.optim as optim
import torchlensmaker as tlm

surface = tlm.surfaces.Sphere(diameter=15, r=tlm.parameter(18))
lens = tlm.BiLens(surface, material="SF10-nd", outer_thickness=1.5)

optics = nn.Sequential(
    tlm.PointSourceAtInfinity(beam_diameter=18.5),
    tlm.Gap(10),
    lens,
    tlm.Gap(30),
    tlm.FocalPoint(),
)

# I think this doesnt even relate to critical angle
lens.optics[2].critical_angle = "drop"


tlm.show(optics, dim=2, sampling={"base": 10})

In [ ]:
### BUG V2
# reproduction without lens

# bug:
# SF10-nd / clamp       : bug
# SF10-nd / drop        : bug maybe different
# SF10-nd / reflect     : unknown
# SF10-nd / nan         : json error

# no bug:
# water-nd / any

import torch
import torch.nn as nn
import torch.optim as optim
import torchlensmaker as tlm

surface = tlm.surfaces.Sphere(diameter=15, r=-18)

optics = nn.Sequential(
    tlm.Offset(
        tlm.Rotate(
            tlm.RaySource(material="SF10-nd"),
            [-20, 0]),
        y=10),
    tlm.Gap(10),
    tlm.RefractiveSurface(surface, material="air", anchors=("origin", "origin"), critical_angle="nan"),
    tlm.Gap(30),
    tlm.FocalPoint(),
)


tlm.show(optics, dim=2, sampling={"base": 10})